# Phân tích bộ dữ liệu `penguins`


## 1. Preparing datasets

In [ ]:
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# --- Thư viện cho Machine Learning ---

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


sys.path.append('../')
from src.utils import plot_label_original, plot_label_distribution, build_decision_tree_penguins, evaluating_decision_tree_class_penguins


PENGUINS_PATH      = '../data/palmer_penguins.csv'
OUTPUT_DIR         = '../outputs/penguins/'
os.makedirs(OUTPUT_DIR + 'charts/', exist_ok=True)

df = pd.read_csv(PENGUINS_PATH)

df = df[['species','bill_length_mm','flipper_length_mm','body_mass_g','sex']]


imp = SimpleImputer(strategy='median')
df[['bill_length_mm','flipper_length_mm','body_mass_g']] = \
    imp.fit_transform(df[['bill_length_mm','flipper_length_mm','body_mass_g']])
df['sex'] = df['sex'].fillna('Unknown')

# One-hot encode sex
X = pd.get_dummies(df.drop('species', axis=1))
y = df['species']

# 1) Vẽ phân phối gốc
plot_label_original(y, OUTPUT_DIR)

# 2) Các tỷ lệ cần chia
ratios = [
    (40, 60),
    (60, 40),
    (80, 20),
    (90, 10),
]

# 3) Chia và gán biến + vẽ phân phối
for train_pct, test_pct in ratios:
    test_size = test_pct / 100
    
    # stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        shuffle=True,
        stratify=y,
        random_state=42
    )
    
    # Gán vào biến toàn cục (tuỳ chọn)
    globals()[f'feature_train_{train_pct}'] = X_train
    globals()[f'label_train_{train_pct}']   = y_train
    globals()[f'feature_test_{test_pct}']   = X_test
    globals()[f'label_test_{test_pct}']     = y_test
    
    # In thông báo và vẽ phân phối
    print(f"-- Split {train_pct}/{test_pct}: train {len(y_train)} | test {len(y_test)}")
    plot_label_distribution(
        y_train, y_test,
        train_pct, test_pct,
        OUTPUT_DIR
    )

Biểu đồ đã được lưu tại: ../outputs/penguins/charts/class_original.png
-- Split 40/60: train 137 | test 207
Biểu đồ đã được lưu tại: ../outputs/penguins/charts/class_distribution_40_60.png
-- Split 60/40: train 206 | test 138
Biểu đồ đã được lưu tại: ../outputs/penguins/charts/class_distribution_60_40.png
-- Split 80/20: train 275 | test 69
Biểu đồ đã được lưu tại: ../outputs/penguins/charts/class_distribution_80_20.png
-- Split 90/10: train 309 | test 35
Biểu đồ đã được lưu tại: ../outputs/penguins/charts/class_distribution_90_10.png


## 2. Building the decision tree classifiers


In [18]:
print("Đang tạo confusion matrix 60/40...")
clf_60_40 = build_decision_tree_penguins(X, feature_train_60, label_train_60, 60, 40, OUTPUT_DIR)

print("Đang tạo confusion matrix 40/60...")
clf_40_60 = build_decision_tree_penguins(X, feature_train_40, label_train_40, 40, 60, OUTPUT_DIR)

print("Đang tạo confusion matrix 80/20...")
clf_80_20 = build_decision_tree_penguins(X, feature_train_80, label_train_80, 80, 20, OUTPUT_DIR)

print("Đang tạo confusion matrix 90/10...")
clf_90_10 = build_decision_tree_penguins(X, feature_train_90, label_train_90, 90, 10, OUTPUT_DIR)

Đang tạo confusion matrix 60/40...
Đã tạo decision tree tại ../outputs/penguins/trees\decision_tree_60_40.png
Đang tạo confusion matrix 40/60...
Đã tạo decision tree tại ../outputs/penguins/trees\decision_tree_40_60.png
Đang tạo confusion matrix 80/20...
Đã tạo decision tree tại ../outputs/penguins/trees\decision_tree_80_20.png
Đang tạo confusion matrix 90/10...
Đã tạo decision tree tại ../outputs/penguins/trees\decision_tree_90_10.png


## 3. Evaluating the decision tree classifiers


In [19]:
print("Đang tạo Classification Report và Confusion Matrix cho test 40")
evaluating_decision_tree_class_penguins(clf_60_40, feature_test_40, label_test_40, 60, 40, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 60")
evaluating_decision_tree_class_penguins(clf_40_60, feature_test_60, label_test_60, 40, 60, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 20")
evaluating_decision_tree_class_penguins(clf_80_20, feature_test_20, label_test_20, 80, 20, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 10")
evaluating_decision_tree_class_penguins(clf_90_10, feature_test_10, label_test_10, 90, 10, OUTPUT_DIR)


Đang tạo Classification Report và Confusion Matrix cho test 40
Classification report saved: ../outputs/penguins/reports/classification_report_60_40.txt
Confusion matrix saved: ../outputs/penguins/matrices\confusion_matrix_60_40.png
Đang tạo Classification Report và Confusion Matrix cho test 60
Classification report saved: ../outputs/penguins/reports/classification_report_40_60.txt
Confusion matrix saved: ../outputs/penguins/matrices\confusion_matrix_40_60.png
Đang tạo Classification Report và Confusion Matrix cho test 20
Classification report saved: ../outputs/penguins/reports/classification_report_80_20.txt
Confusion matrix saved: ../outputs/penguins/matrices\confusion_matrix_80_20.png
Đang tạo Classification Report và Confusion Matrix cho test 10
Classification report saved: ../outputs/penguins/reports/classification_report_90_10.txt
Confusion matrix saved: ../outputs/penguins/matrices\confusion_matrix_90_10.png


## 4. The depth and accuracy of a decision tree


In [20]:


depths = [None, 2, 3, 4, 5, 6, 7]
accuracies = []

output_dir = '../outputs/penguins/acc/'
os.makedirs(output_dir, exist_ok=True)
results = []

for depth in depths:
    str_depth = f" với độ sâu là {depth} " if depth is not None else ""
    print(f"Đang tạo cây quyết định{str_depth}...")

    clf = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=42)
    clf.fit(feature_train_80, label_train_80)
    y_pred = clf.predict(feature_test_20)
    acc = accuracy_score(label_test_20, y_pred)
    accuracies.append(acc)

    results.append({
        "max_depth": str(depth),
        "Accuracy": acc
    })

    # Vẽ và lưu cây quyết định
    dot_data = export_graphviz(
        clf,
        out_file=None,
        feature_names=X.columns,
        class_names=clf.classes_,
        filled=True,
        rounded=True,
        special_characters=True
    )
    graph = graphviz.Source(dot_data)
    filename = f"{output_dir}tree_depth_{depth if depth is not None else 'None'}"
    graph.render(filename=filename, format="pdf", cleanup=True)
    print(f"Đã lưu cây quyết định tại {filename}.pdf")

# Tạo bảng kết quả accuracy
print("Đang tạo bảng accuracy_score...")
accuracy_dict = {entry["max_depth"]: entry["Accuracy"] for entry in results}
results_df = pd.DataFrame([accuracy_dict], index=["Accuracy"])
results_df.columns.name = "max_depth"
results_df.to_csv(f"{output_dir}accuracy_score.csv", index=True, index_label="Type")
print(f"Đã lưu bảng accuracy_score tại {output_dir}accuracy_score.csv")

# Vẽ biểu đồ
plt.figure(figsize=(8, 5))
plt.plot([str(d) for d in depths], accuracies, marker='o')
plt.title("Accuracy vs Max Depth (Penguins 80/20 Split)")
plt.xlabel("max_depth")
plt.ylabel("Accuracy")
plt.grid(True)
plt.savefig(f"{output_dir}accuracy_vs_depth.png")
plt.close()
print(f"Đã lưu biểu đồ accuracy_score tại {output_dir}accuracy_vs_depth.png")


Đang tạo cây quyết định...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_None.pdf
Đang tạo cây quyết định với độ sâu là 2 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_2.pdf
Đang tạo cây quyết định với độ sâu là 3 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_3.pdf
Đang tạo cây quyết định với độ sâu là 4 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_4.pdf
Đang tạo cây quyết định với độ sâu là 5 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_5.pdf
Đang tạo cây quyết định với độ sâu là 6 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_6.pdf
Đang tạo cây quyết định với độ sâu là 7 ...
Đã lưu cây quyết định tại ../outputs/penguins/acc/tree_depth_7.pdf
Đang tạo bảng accuracy_score...
Đã lưu bảng accuracy_score tại ../outputs/penguins/acc/accuracy_score.csv
Đã lưu biểu đồ accuracy_score tại ../outputs/penguins/acc/accuracy_vs_depth.png
